### ***Import Libraries***

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_scheduler
from datasets import load_metric

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# %matplotlib inline
# %config InlineBackend.figure_format='retina'
# sns.set(style='whitegrid', palette='muted', font_scale=1.2)
# rcParams['figure.figsize'] = 16, 10

### ***Load Model***

In [2]:
tokenizer = AutoTokenizer.from_pretrained("abdalrahmanshahrour/auto-arabic-summarization")

model = AutoModelForSeq2SeqLM.from_pretrained("abdalrahmanshahrour/auto-arabic-summarization")

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/557M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### ***Load Dataset***

In [3]:
data = pd.read_csv("/kaggle/input/arabicsummarization/summarizdataset.csv")

data.head()

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


In [4]:
data.drop('type',inplace=True,axis=1)

In [5]:
data.drop('text',inplace=True,axis=1)

In [6]:
data

,Processed Text,summarizer
0,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...
...,...,...
8373,تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...,وكانت انعقدت عشية أمس في وزارة الشؤون الاجتماع...
8374,كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...,وأكدا ان تكلفة العلم لم تتجاوز 3 آلاف دينار مق...
8375,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...,كما تمّ إلقاء القبض على عنصر رابع (عمره 32 سنة...
8376,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...,وتأتي هذه الخطوة على خلفية ما اعتبره أهالي هذه...


### ***Data Preprocessing***

In [7]:
# Data Preprocessing
# First Checking 'duplicated rows' and removing them if they are exist
dupl = data[data.duplicated()]
if len(dupl)>0:
    data=data.drop_duplicates()
    print(len(dupl))
    
# !pip install nltk
import re
import nltk
### from nltk.stem.isri import ISRIStemmer
### st = ISRIStemmer()

# Data cleaning
def clean_text(text):
    # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A \u0660-\u0669 0-9 \s ( ) : . ، ؛ ]+','',text)
    
    # Remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text = re.sub(r'\.{2,}', '', text)

    return text

# Data preprocessing
def pre_process(text):
    tokens = nltk.word_tokenize(text)
    stop_words = ["و", "في", "من", "على", "إلى", "عن", "فيه", "عليه", "هو", "هي"]
    text = " ".join([word for word in tokens if word not in stop_words])

    return text

data['Processed Text'] = data['Processed Text'].apply(clean_text)

print(data['Processed Text'])

# Apply the clean_text function to the content column
data['Processed Text'] = data['Processed Text'].apply(clean_text)
print(data['Processed Text'])


324


/tmp/ipykernel_84/1698713258.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Processed Text'] = data['Processed Text'].apply(clean_text)


0       اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1       تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2       احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3       شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4       نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...
                              ...                        
8373    تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...
8374    كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...
8375    فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...
8376    قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...
8377    تمكنت وحدات الحرس الوطني بمحطه الاستخلاص ببرج ...
Name: Processed Text, Length: 8054, dtype: object
0       اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...
1       تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...
2       احتضن جناح تونس القريه الدوليه للافلام بمدينه ...
3       شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...
4       نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...
                      

/tmp/ipykernel_84/1698713258.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Processed Text'] = data['Processed Text'].apply(clean_text)


In [8]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('arabic'))
preprocessed_text=[]
def preprocess_arabic_Remove_stop(text):
    
    # Tokenize text into words
    words = nltk.word_tokenize(text)
    
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    
    # Stem the words
    #words = [stemmer.stem(word) for word in words]
    
    # Join the words back into a single string
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text

data['Processed Text'] = data['Processed Text'].apply(preprocess_arabic_Remove_stop)
data['Processed Text'] = data['Processed Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/tmp/ipykernel_84/1379311527.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Processed Text'] = data['Processed Text'].apply(preprocess_arabic_Remove_stop)
/tmp/ipykernel_84/1379311527.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Processed Text'] = data['Processed Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))


,Processed Text,summarizer
0,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...


In [9]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=aa30ab90dea770aa42d5d9d1089109268f9d17ce3330bcf59170b8cbf5385a2a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [10]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data=data,
        tokenizer=tokenizer,
        text_max_token_len = 800,
        summary_max_token_len = 150
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['Processed Text']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summarizer'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [11]:
# Data Spliting into train and val  
df_train, df_test = train_test_split(data, test_size=0.2, random_state=42)

train_dataset = SummaryDataset(data=df_train)
test_dataset = SummaryDataset(data=df_test)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)
eval_dataloader = DataLoader(test_dataset, batch_size=2)

In [12]:
df_train.to_csv('train.csv')
df_test.to_csv('val.csv')

In [13]:
df_train

,Processed Text,summarizer
6525,تمكنت وحدات الحرس الوطني بجهه بوحجله ولايه الق...,بالتحري معها حول دوافعها من عملية اختطاف وتكبي...
1409,قال كبير اداره الرئيس الاميركي دونالد ترامب ال...,\nقال مسؤولٌ كبير في إدارة الرئيس الأميركي دون...
826,اعلنت وزاره الداخليه المغربيه اليوم الاربعاء ا...,\nأعلنت وزارة الداخلية المغربية اليوم الأربعاء...
1848,رفض الرئيس السوري الاسد السماح لوفود تابعه لمن...,"وأوضح في مقابلة مع إذاعة ""Europe 1"" وقناة ""TF1..."
1637,انتزع مرشح فرنسا الثائره للانتخابات الفرنسيه ج...,"\nانتزع مرشح ""فرنسا الثائرة"" للإنتخابات الفرنس..."
...,...,...
5485,افاد المدير العام للامتحانات بوزارهالتربيه عمر...,ويضبط المعدل النهائي لامتحان الباكالوريا بالنس...
5654,بينت فوزيه مبروك المتفقده الجهويه للصحه بسوسه ...,\nبينت فوزية مبروك المتفقدة الجهوية للصحة بسوس...
874,قال الحاج ابو يوسف الحمراني القائد بفصائل المق...,\nقال الحاج ابو يوسف الحمراني، القائد بفصائل ا...
7914,شهدت مدينه بئر بن خليفه ولايه صفاقس اليوم الاث...,ويطالب أهالي المدينة بحق الجهة في التنمية وتحس...


In [14]:
df_test

,Processed Text,summarizer
2972,تدور اليوم عمليه سحب القرعه الخاصه بمرحله التت...,\nتدور اليوم عملية سحب القرعة الخاصة بمرحلة ال...
6552,اكد رئيس الحكومه يوسف الشاهد ضروره تشجيع المبا...,وشدد رئيس الحكومة على ضرورة تكوين جيل جديد من ...
2065,افاد سفيان طوبال تصريح اعلامي اليوم السبت ختام...,وأضاف طوبال أن 37 نائبا صوتوا إثر ذلك على احتر...
6201,اكدت امنه القلالي مديره مكتب تونس لمنظمه هيومن...,وأضافت أن الدعوة العامة للمظاهرة التي ستجري غ...
3572,اعلن الفرنسي باسكال جانان انسحابه مهامه كمدير ...,\nأعلن الفرنسي باسكال جانان عن إنسحابه من مهام...
...,...,...
8150,اعلن الديوان الوطني للحمايه المدنيه حصيله تدخل...,"كما تمّت معاينة جثة مواطن على مستوى معهد ""ابن ..."
4625,اكد الامين العام المساعد بالاتحاد العام التونس...,"ورفع المحتجون في هذه المسيرة التي وصفوها ب""يوم..."
6761,قال ابراهيم الميساوي رئيس الجمعيه التونسيه لمك...,\nقال ابراهيم الميساوي رئيس الجمعية التونسية ل...
6602,تحدث عمر بن مسعود مشرف اداره المرحله الاعداديه...,"وختم عمر بن مسعود بالقول ""صفحات التواصل الاجتم..."


### ***Train mbart Model*** 

In [15]:
num_epochs = 4

num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         print(predictions)
#         print(batch["labels"])
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5-Arabic.pth')

    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/12888 [00:00<?, ?it/s]

epoch: 1 -- loss: 1.424535870552063
epoch: 2 -- loss: 0.9762586951255798
epoch: 3 -- loss: 1.1869345903396606
epoch: 4 -- loss: 0.6154674291610718


In [16]:
metric= load_metric("rouge")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'rouge1': AggregateScore(low=Score(precision=0.29140854541692557, recall=0.29140854541692557, fmeasure=0.29140854541692557), mid=Score(precision=0.2980715911442171, recall=0.2980715911442171, fmeasure=0.2980715911442171), high=Score(precision=0.3044664804469278, recall=0.3044664804469278, fmeasure=0.3044664804469278)),
 'rouge2': AggregateScore(low=Score(precision=0.20562825207570362, recall=0.20562825207570362, fmeasure=0.20562825207570362), mid=Score(precision=0.21046996529730488, recall=0.21046996529730488, fmeasure=0.21046996529730488), high=Score(precision=0.21572036210782328, recall=0.21572036210782328, fmeasure=0.21572036210782328)),
 'rougeL': AggregateScore(low=Score(precision=0.2733579557210838, recall=0.2733579557210838, fmeasure=0.2733579557210838), mid=Score(precision=0.2792447755017583, recall=0.2792447755017583, fmeasure=0.2792447755017583), high=Score(precision=0.2848806124560309, recall=0.2848806124560309, fmeasure=0.2848806124560309)),
 'rougeLsum': AggregateScore(lo

In [17]:
def summarizeText(text, model=model):
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = model.generate(
        input_ids=text_encoding['input_ids'].to(device),
        attention_mask=text_encoding['attention_mask'].to(device),
        max_length=150,
        num_beams=4,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )    

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)


In [18]:
path_input = '/kaggle/working/val.csv'

# Read the CSV file
dataset = pd.read_csv(path_input)

dataset.head()

,Unnamed: 0,Processed Text,summarizer
0,2972,تدور اليوم عمليه سحب القرعه الخاصه بمرحله التت...,\nتدور اليوم عملية سحب القرعة الخاصة بمرحلة ال...
1,6552,اكد رئيس الحكومه يوسف الشاهد ضروره تشجيع المبا...,وشدد رئيس الحكومة على ضرورة تكوين جيل جديد من ...
2,2065,افاد سفيان طوبال تصريح اعلامي اليوم السبت ختام...,وأضاف طوبال أن 37 نائبا صوتوا إثر ذلك على احتر...
3,6201,اكدت امنه القلالي مديره مكتب تونس لمنظمه هيومن...,وأضافت أن الدعوة العامة للمظاهرة التي ستجري غ...
4,3572,اعلن الفرنسي باسكال جانان انسحابه مهامه كمدير ...,\nأعلن الفرنسي باسكال جانان عن إنسحابه من مهام...


In [19]:
dataset.drop('Unnamed: 0',inplace=True,axis=1)

In [20]:
dataset

,Processed Text,summarizer
0,تدور اليوم عمليه سحب القرعه الخاصه بمرحله التت...,\nتدور اليوم عملية سحب القرعة الخاصة بمرحلة ال...
1,اكد رئيس الحكومه يوسف الشاهد ضروره تشجيع المبا...,وشدد رئيس الحكومة على ضرورة تكوين جيل جديد من ...
2,افاد سفيان طوبال تصريح اعلامي اليوم السبت ختام...,وأضاف طوبال أن 37 نائبا صوتوا إثر ذلك على احتر...
3,اكدت امنه القلالي مديره مكتب تونس لمنظمه هيومن...,وأضافت أن الدعوة العامة للمظاهرة التي ستجري غ...
4,اعلن الفرنسي باسكال جانان انسحابه مهامه كمدير ...,\nأعلن الفرنسي باسكال جانان عن إنسحابه من مهام...
...,...,...
1606,اعلن الديوان الوطني للحمايه المدنيه حصيله تدخل...,"كما تمّت معاينة جثة مواطن على مستوى معهد ""ابن ..."
1607,اكد الامين العام المساعد بالاتحاد العام التونس...,"ورفع المحتجون في هذه المسيرة التي وصفوها ب""يوم..."
1608,قال ابراهيم الميساوي رئيس الجمعيه التونسيه لمك...,\nقال ابراهيم الميساوي رئيس الجمعية التونسية ل...
1609,تحدث عمر بن مسعود مشرف اداره المرحله الاعداديه...,"وختم عمر بن مسعود بالقول ""صفحات التواصل الاجتم..."


In [21]:
# Data Preprocessing
# First Checking 'duplicated rows' and removing them if they are exist
dupl = dataset[dataset.duplicated()]
if len(dupl)>0:
    dataset=dataset.drop_duplicates()
    print(len(dupl))
    
    # Second Checking 'empty cells' and removing them if they are exist
    # Here we deal with empty cells (either removing them Or setting them with default data) but we won't do that because data is cleaned

    # Third Cleaning data from (Arabic and English digits, special characters, and extra spaces)
import re
def clean_text(text):

        # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A\u0660 - \0669\s]+', '', text)

        # Remove "English digits"
    text = re.sub('\d+', '', text)

        # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply the clean_text function to the content column
dataset['Processed Text'] = dataset['Processed Text'].apply(clean_text)
dataset['Processed Text']

0       تدور اليوم عمليه سحب القرعه الخاصه بمرحله التت...
1       اكد رئيس الحكومه يوسف الشاهد ضروره تشجيع المبا...
2       افاد سفيان طوبال تصريح اعلامي اليوم السبت ختام...
3       اكدت امنه القلالي مديره مكتب تونس لمنظمه هيومن...
4       اعلن الفرنسي باسكال جانان انسحابه مهامه كمدير ...
                              ...                        
1606    اعلن الديوان الوطني للحمايه المدنيه حصيله تدخل...
1607    اكد الامين العام المساعد بالاتحاد العام التونس...
1608    قال ابراهيم الميساوي رئيس الجمعيه التونسيه لمك...
1609    تحدث عمر بن مسعود مشرف اداره المرحله الاعداديه...
1610    عمد عشيه اليوم الجمعه كهل يبلغ العمر سنه قاطن ...
Name: Processed Text, Length: 1611, dtype: object

In [22]:
import pandas as pd

# Function to apply the model to each instance
def apply_model(instance):
    # Apply your model to the instance and return the output
    output = summarizeText(instance)
    return output

# Create an empty list to store the model outputs
model_outputs = []

# Iterate over the dataframe and apply the model to each instance
for index, row in tqdm(dataset.iterrows()):
    instance = row['Processed Text']  # Assuming 'text' is the column containing the input data
    output = apply_model(instance)
    model_outputs.append(output)

# Add the model outputs as a new column to the dataframe
dataset['summarizer'] = model_outputs

# Display the updated dataframe
print(dataset['summarizer'])


0it [00:00, ?it/s]

0       تدور اليوم عملية سحب القرعة الخاصة بمرحلة التت...
1       أكد رئيس الحكومة يوسف الشاهد على ضرورة تشجيع ا...
2       كما تم التوافق على ترشيح امرأة من نواب نداء تو...
3       وأضافت أن هذه الحملة بدأت في عدد من الجهات الت...
4       أعلن الفرنسي بأسره جانان انسحابه من مهامه كمدي...
                              ...                        
1606    وأفاد الديوان، في بلاغ نشرته وزارة الداخلية ال...
1607    وأكد في المقابل، أن الاتحاد "بالرغم من مسانده ...
1608    قال ابراهيم الميساوي رئيس الجمعية التونسية لمك...
1609    وختم عمر بن مسعود بالقول إن صفحات التواصل الاج...
1610    عمد عشية اليوم الجمعة، كهل يبلغ من العمر 15 سن...
Name: summarizer, Length: 1611, dtype: object


In [23]:
#Data Preprocessing
    #First Checking 'duplicated rows' and removing them if they are exist
dupl = data[data.duplicated()]
if len(dupl)>0:
    data=data.drop_duplicates()
    print(len(dupl))
    
    #Second Checking 'empty cells' and removing them if they are exist
    #Here we deal with empty cells (either removing them Or setting them with default data) but we won't do that because data is cleaned

    #Third Cleaning data from (Arabic and English digits, special characters, and extra spaces)
import re
def clean_text(text):

        # Remove special characters and punctuation and "Arabic digits"
    text = re.sub(r'[^\u0621-\u064A\u0660 - \0669\s]+', '', text)

        # Remove "English digits"
    ##text = re.sub('\d+', '', text)

        # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply the clean_text function to the content column
dataset['summarizer'] = dataset['summarizer'].apply(clean_text)
print(dataset['summarizer'])

0       تدور اليوم عملية سحب القرعة الخاصة بمرحلة التت...
1       أكد رئيس الحكومة يوسف الشاهد على ضرورة تشجيع ا...
2       كما تم التوافق على ترشيح امرأة من نواب نداء تو...
3       وأضافت أن هذه الحملة بدأت في عدد من الجهات الت...
4       أعلن الفرنسي بأسره جانان انسحابه من مهامه كمدي...
                              ...                        
1606    وأفاد الديوان في بلاغ نشرته وزارة الداخلية الي...
1607    وأكد في المقابل أن الاتحاد بالرغم من مسانده قر...
1608    قال ابراهيم الميساوي رئيس الجمعية التونسية لمك...
1609    وختم عمر بن مسعود بالقول إن صفحات التواصل الاج...
1610    عمد عشية اليوم الجمعة كهل يبلغ من العمر سنة قا...
Name: summarizer, Length: 1611, dtype: object
